# Deep Learning Kullanarak Spam E-posta Tespit Etme

### Gerekli Kütüphaneleri import ediyoruz

In [ ]:
# Sayısal hesaplamalar için (Matris işlemleri)
import numpy as np

# Veri manipülasyonu ve analizi için
import pandas as pd

# Makine öğrenimi ve derin öğrenme modelleri
# oluşturmak, eğitmek ve değerlendirmek için
import tensorflow as tf

# Veri görselleştirmesi için
import matplotlib.pyplot as plt

# Düzenli ifadeler (regular expressions) 
# kullanarak metin işleme işlemleri için.
import re

# İşletim sistemleri işlemleri için 
import os

# Kategorik verileri, sayısal değerlere 
# dönüştürmek için
from sklearn.preprocessing import LabelEncoder

# TensorFlow Hub kütüphanesi, önceden 
# eğitilmiş model bileşenlerini ve gömme (embedding)
# vektörlerini kullanmak için.
import tensorflow_hub as hub

# metin verileri üzerinde özel işlemler ve
# modeller oluşturmak için kullanılır.
import tensorflow_text as text

### Dataseti yüklüyoruz

In [2]:
np.set_printoptions(precision=3,suppress=True)

# Verileri okunabilir hale getiriyoruz

In [3]:
# pd kullanarak dataset'imizi okuyoruz
data = pd.read_csv("Dataset.csv")

# head kullanarak birkaç adet veriyi gösterelim
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing İşlemleri

In [4]:
# np.set_printoptions(precision=3,suppress=True)
# Bu işlemden sonra gereksiz satırlar da 
# datasete eklendi. Onları çıkarıyoruz.
df =  pd.DataFrame()
df[['label','text']] = data[['v1','v2']]

In [5]:
df.head()
# Birkaç satırı görelim.

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.describe()
# Bir adet boş kayıt bırakacağımız için toplam kayıt 5567

,label,text
count,5568,5567
unique,2,5161
top,ham,"Sorry, I'll call later"
freq,4821,30


In [7]:
# label sütununa göre gruplama yapıyoruz
df.groupby('label').describe()

text                                                               
      count unique                                                top freq
label                                                                     
ham    4820   4506                             Sorry, I'll call later   30
spam    747    655  Please call our customer service representativ...    4

In [8]:
# eksik (NaN) değerlerin sayısını hesaplıyoruz.
# Sonuçta da label indeksinde 0
# text indeksinde de 1 adet NaN değer var.
df.isna().sum()

label    0
text     1
dtype: int64

In [9]:
# null değerleri çıkarıyoruz.
# Böylece label indexinde de 
# text indexinde de 0 adet NaN değer kaldı.
df.dropna(inplace= True)
df.isna().sum()

label    0
text     0
dtype: int64

In [10]:
# Birkaç örneği inceleyelim.
df.head() 

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
# Mesajlardaki
# En küçük sayıda karakterleri
min_len = df['text'].min()
print(len(min_len))\

# En büyük sayıda karakterleri buluyoruz.
max_len = df['text'].max()
print(len(max_len))

21
81


In [12]:
# Metinlere bakalım.
df['text']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
5       FreeMsg Hey there darling it's been 3 week's n...
6       Even my brother is not like to speak with me. ...
7       As per your request 'Melle Melle (Oru Minnamin...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
10      I'm gonna be home soon and i don't want to tal...
11      SIX chances to win CASH! From 100 to 20,000 po...
12      URGENT! You have won a 1 week FREE membership ...
13      I've been searching for the right words to tha...
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
15      XXXMobileMovieClub: To use your credit, click ...
16                             Oh k...i'm watching here:)
17      Eh u r

In [13]:
# Text indexini  ----> X
# Label indexini ----> Y
# olarak assign ediyoruz.
x = df['text'].copy()
y = df['label'].copy()

# Ardından X ve Y'nin birkaç
# örneğine bakalım.
print(x.head())
print(y.head())

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: text, dtype: object
0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object


In [14]:
# Dataset'in label yani y kısmını encode edelim.

In [15]:
y.head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object

In [16]:
# ham  ===> 0
# spam ===> 1
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [17]:
y_encoded = pd.DataFrame(y_enc)


In [18]:
# Datasetin y indexinin yeni haline bakalım.
y_encoded.head()

,0
0,0
1,0
2,1
3,0
4,0


In [19]:
# y'leri encode ettik. Şimdi X'lere bakalım.

In [20]:
# X' in boyutune bakalım
x.shape

(5567,)

In [22]:
# y_encoded DataFrame'inin ilk birkaç satırını gösterir. 
y_encoded.head()

,label
0,0
1,0
2,1
3,0
4,0


In [23]:
# X'i Pandas DataFrame'e dönüştürelim.
x_df = pd.DataFrame(x)

In [24]:
# Örneklerine bakalım.
x_df.head()

,text
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
# Encode ettiğimiz X ve Y'lerin asıl isimlerini
# X ve Y olarak düzenleyelim.
X = x_df.copy()
Y = y_encoded.copy()

In [26]:
# X örnekleri
X.head()

,text
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
# Y örnekleri
Y.head()

,label
0,0
1,0
2,1
3,0
4,0


In [28]:
print(X.shape)
print(Y.shape)
# X'in boyutlarıyla, Y'nin boyutu aynı

(5567, 1)
(5567, 1)


In [29]:
# Burada train edilecek data ile 
# test edilecek datayı ayırıyoruz.
# Ve ilgili değerlere assign ediyoruz.
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain,ytest = train_test_split(X,Y,test_size=0.2)


In [30]:
# X'in train edilecek data boyutu
print(xtrain.shape)

# X'in test edilecek data boyutu
print(xtest.shape)

# Y'nin train edilecek data boyutu
print(ytrain.shape)

# Y'in test edilecek data boyutu
print(ytest.shape)

(4453, 1)
(1114, 1)
(4453, 1)
(1114, 1)


In [31]:
# BERT model 1. preprocessor and encoder are trained here

# önceden eğitilmiş bir BERT modelinin ön işleme katmanını yüklüyoruz.
# BERT modelleri, giriş metni üzerinde çeşitli işlemler gerçekleştirerek
# metni modelin anlayabileceği bir gömme (embedding) formatına dönüştürür.
# Bu ön işleme katmanı, metin verilerini BERT modeline uygun hale getirmek için kullanılır.
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# önceden eğitilmiş bir BERT modelinin kodlama katmanını (encoder) yükler.
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
    trainable=True)

In [32]:
# Bu fonksiyon, bir giriş cümlesini alır, bu cümleyi önceden eğitilmiş
# BERT modelinin ön işleme katmanından geçirir (preprocessor), ardından
# kodlama katmanından geçirir (encoder), ve son olarak kodlanmış cümlenin
# gömme (embedding) temsilini döndürür 
def sentence_embedding(sentence):
    preprocessed_input = preprocessor(sentence)
    encoded_input = encoder(preprocessed_input)
    return encoded_input['pooled_output']

In [ ]:
# Aynı şekilde burada da bir örnek yapıyoruz.
# önce prepocessor katmanından geçiyor.
# ardından encode ediliyor.
# sonra da embedded olarak return ediyor.
sentence_embedding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

In [34]:
# Bert Layer

text_input = tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text = preprocessor(text_input)
output = encoder(preprocessed_text)

In [ ]:
# Neural Network Layer
# Bu satır, bir dropout katmanını (Dropout) ekler.
# Dropout, ağın öğrenirken aşırı uyum (overfitting)
# yapmasını önlemek için kullanılan bir regularizasyon tekniğidir. 
l = tf.keras.layers.Dropout(0.1, name="dropout") (output['pooled_output'])

l = tf.keras.layers.Dense(1, activation='sigmoid',name = 'output')(l)

# Sonuç olarak, bu iki satır, bir sinir ağı modelinin bir kısmını oluşturur.
# Bu modelin girişi, önceden eğitilmiş bir BERT modeli tarafından üretilen 
# gömme temsilleridir ve çıkışı ise ikili sınıflandırma yapmak için kullanılan
# bir yoğun katmandır. Dropout, modelin genelleştirmesini artırmaya yardımcı 
# olurken, yoğun katman sınıflandırma işlemini gerçekleştirir.

In [ ]:
# use i/p and op for model
# Keras modelinin giriş çıkışlarını belirtiyoruz.
# böylelikle giriş ve çıkışları birbirine bağlıyoruz.
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [37]:
# Modeli daha iyi anlamamız için özet
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
_______________________________________________________________________________________

In [ ]:
# Model için metrikleri ekliyoruz.
METRICS = [
    tf.keras.metrics.Accuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall")
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics= METRICS
    )

In [39]:
# modeli train ediyoruz.
model.fit(xtrain, ytrain, batch_size= 64, epochs=1)